# Estimating Damage from Natural Disasters in the USA

## Install the Python requirements below.

In [ ]:
# Step 0: Download this notebook and upload it to Google colab

# Step 1: Download the Python code from Github
wget -q -nc https://github.com/ss-github-code/noaa_storm_analysis/archive/refs/heads/main.zip
unzip -q main.zip noaa_storm_analysis-main/*; mv noaa_storm_analysis-main/* .;rm -r noaa_storm_analysis-main
pip install -r requirements.txt

## Import the required packages.

In [21]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
import altair as alt
import json
import requests
import http.client

import datetime
import os

from io import StringIO
from datetime import date
from datetime import timedelta
from urllib.error import HTTPError
from ipywidgets.widgets.widget_selection import Dropdown
from IPython.display import Javascript

from IPython.display import display

from app_df import get_counties_json, get_storm_data, get_list_years, STORM_CATEGORIES
from notebooks.data_constants import ZONE_COUNTY_CORR_URL

alt.data_transformers.disable_max_rows();
alt.renderers.enable('default', embed_options={'actions': False}); # hide the option to export chart as png

# Constants
WILDFIRE_DATA_URL = 'https://raw.githubusercontent.com/ss-github-code/noaa_storm_analysis/main/data/wildfires/'

## Setup the user interface to explore economic damage from storms using Ipywidgets

The user interface allows you to select a year to explore the damage caused by the natural disasters in that year using a choropleth map. You can also select one of the 3 major types of storms; they are Tropical Cyclones/Floods, Severe Local Storms, and Wildfires/Droughts.

Additionally, you can click on an event displayed in the scatter-plot below to explore the independent variables from the affected county like population and economic activity that could be used to predict the total damage caused by the natural disasters in the USA.

In [15]:
# Uncomment this line if you are viewing using Google colab
# display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

box_layout_plotly = widgets.Layout(display='flex', flex_flow='row', overflow='scroll',
                            border='2px solid black',
                            #width='100%',
                            height='500px')
box_layout_altair = widgets.Layout(display='flex', flex_flow='row', overflow='scroll',
                            border='2px solid black',
                            #width='100%',
                            height='650px')

outputPlotly = widgets.Output(layout=box_layout_plotly)
outputAltair = widgets.Output(layout=box_layout_altair)

res_layout = widgets.Layout(align_content='flex-start')
res_output = widgets.VBox([outputPlotly, outputAltair], layout=res_layout )
with outputPlotly:
    print('Please select a year to view the economic damage from the storms for that year')

def clicked(b):
    if b['type'] == 'change' and b['name'] == 'value':
        if type(b['owner']) == Dropdown:    # if year changed, clear both plots
            outputPlotly.clear_output()
            outputAltair.clear_output()        
            if b['new'] == '': # if year == ''
                with outputPlotly:
                    print('Please select a year to view the economic damage from the storms for that year')
                return

            year = b['new'] # resolve issue with Safari browser take the year value from the object itself
            layers = layersRadio.value
        else: # changed the radio button
            outputAltair.clear_output() # only clear the scatter plot   
            year = selYearDrop.value
            if year == '':
                return
            layers = layersRadio.value
    
    with outputPlotly:
        title_event = 'storms'
        df_map, _, _ = get_storm_data(year)
        map_columns = df_map.columns
        if layers != 'All events':
            df_map = df_map[df_map[layers]==True]
            title_event = layers

        if df_map.shape[0] == 0:
            df_map = pd.DataFrame([[0]*len(map_columns)], columns=map_columns.tolist())

        counties = get_counties_json()

        if layers == 'All events':
            hover_data_dict = {'FIPS': False, 'ALL_EVENTS' : True, 'TOTAL_DAMAGE': True}
            color_col = 'TOTAL_DAMAGE'
        else:
            for i, k in enumerate(STORM_CATEGORIES.keys()):
                if k == layers:
                    df_map = df_map[['FIPS', 'NAME', 'EVENT_TYPE_'+str(i), 'TYPE_'+str(i)+'_DAMAGE']]
                    df_map.rename(columns={'EVENT_TYPE_'+str(i) : 'EVENTS', 'TYPE_'+str(i)+'_DAMAGE': 'DAMAGES'}, inplace=True)
                    hover_data_dict = {'FIPS': False, 'EVENTS' : True, 'DAMAGES': True}
                    color_col = 'DAMAGES'
                    break
    
        fig = px.choropleth(df_map, 
                        geojson = counties,    # use the geo info for counties
                        locations='FIPS',      # location is based on FIPS code
                        hover_name = 'NAME',   # this has both the county and state name
                        hover_data = hover_data_dict,
                        color = color_col,     # TOTAL_DAMAGE or DAMAGE
                        scope = 'usa') # set the scope to usa to automatically configure the 
                                       # map to display USA-centric data in an appropriate projection.
        fig.update_layout(title_text = f'<b>Total damage in $ from {title_event} in year {year}</b>',
                          title_font_size=13, title_x=0.5, title_xanchor='center') # match font-size and location with Altair
        fig.show()

    with outputAltair:
        print('Click on an event in the graph below to see the statistics for the affected county;')
        print('Click on the legend to select events of a particular type.')
        
        # get the storm data from the database
        _, df_counties, df_county_details = get_storm_data(year)
        titleStr = "All events"
        if layers != 'All events':
            df_counties = df_counties[df_counties[layers]==True]
            titleStr = layers

        temp_df = df_counties[['EVENT_TYPE', 'NAME', 'FIPS', 'TOTAL_DAMAGE', 'EVENT_DATE']]
        if temp_df.shape[0]:
            selection = alt.selection_multi(fields=['EVENT_TYPE'], bind='legend') # click on the legend
            brush = alt.selection(type='single', empty='none', fields=['FIPS'])   # click on an event

            chart = alt.Chart(temp_df).mark_circle().encode(
                x = alt.X('EVENT_DATE:T', axis=alt.Axis(grid=False)),
                y = alt.Y('TOTAL_DAMAGE:Q', title = 'Total damage in $(log)', scale=alt.Scale(type='log'), axis=alt.Axis(grid=False)),
                size = alt.Size('TOTAL_DAMAGE:Q',scale=alt.Scale(type='log')),
                color = alt.condition(brush, alt.value('black'), 'EVENT_TYPE:N'),
                opacity=alt.condition(selection, alt.value(1), alt.value(0)),
                tooltip = [alt.Tooltip('NAME'), 
                           alt.Tooltip('TOTAL_DAMAGE', format=','), 
                           alt.Tooltip('EVENT_TYPE'),
                           alt.Tooltip('EVENT_DATE')]
            ).add_selection(
                selection
            ).add_selection(
                brush
            ).properties(
                title = alt.TitleParams([f'Total damage caused by {titleStr} (in $) in year {str(year)}',' ']),
                height = 500,
                width = 450
            )

            text = alt.Chart(df_county_details).mark_text(align='left', dy=-5, limit=100).encode(
                x=alt.value(0),
                y=alt.Y('row_number:O', axis=alt.Axis(labels=False, grid=True, title=None, ticks=False, domain=False)),
            ).transform_window(
                row_number='row_number()'
            ).transform_filter(
                brush
            ).add_selection(
                alt.selection_single() # makes the tooltip to work!
            )
            # Data Tables
            desc = text.encode(
                    text='Features:N',
                    tooltip=[alt.Tooltip('NAME'), alt.Tooltip('Features', title='Feature')]
                ).properties(title=alt.TitleParams('Features', anchor='start'), width=100)
            vals = text.encode(
                    text='DATA_COL:N',
                    tooltip=[alt.Tooltip('NAME'), alt.Tooltip('Features', title='Feature'), alt.Tooltip('DATA_COL', title='Value')]
                ).properties(title=alt.TitleParams('Values', anchor='start'), width=100)
            chart = alt.hconcat(chart, desc, vals).configure_view(strokeOpacity=0)        
        else:
            chart = alt.Chart(df_counties).mark_point(

            ).properties(
                title = alt.TitleParams(['No events of the type ' + layers + ' found for the year ' + str(year), ' '])
            )
        chart.display()

options = ['']
options.extend([i for i in get_list_years()])
selYearDrop = widgets.Dropdown(
    description='Select year',
    options=options,
    value='', 
    disabled=False
)
option_categories = ['All events']
for k, _ in STORM_CATEGORIES.items():
    option_categories.append(k)
layersRadio = widgets.RadioButtons(description="Layers",options=option_categories)

selYearDrop.observe(clicked, names=['value']) # filter only once, specify value in observe
layersRadio.observe(clicked, names=['value'])

list_widgets = [widgets.HBox([selYearDrop, layersRadio])]
accordion = widgets.Accordion(children=list_widgets)
accordion.set_title(0,"Explore NOAA Storm Damage (in $) Data")
display(accordion,res_output)

Accordion(children=(HBox(children=(Dropdown(description='Select year', options=('', '2000', '2001', '2002', '2…

## Setup the user interface to explore drought conditions and historic weather data for the counties affected by wildfires.

The user interface allows you to select a year to explore the drought and weather conditions in the county leading up to the wildfire. The drought conditions are displayed in a choropleth map and the weather details are displayed in line and bar graphs below.

In [25]:
# Uncomment this line if you are viewing using Google colab
# display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

box_map_layout = widgets.Layout(display='flex', overflow='scroll',
                            border='2px solid black',
                            #width='100%',
                            height='600px')
box_charts_layout = widgets.Layout(display='flex', overflow='scroll',
                            border='2px solid black',
                            #width='100%',
                            height='900px')

output_map = widgets.Output(layout=box_map_layout)
output_charts = widgets.Output(layout=box_charts_layout)

res_layout = widgets.Layout(align_content='flex-start')
res_output = widgets.VBox([output_map, output_charts], layout=res_layout)
                           

df_counties = None
df_zone_county = pd.read_csv(ZONE_COUNTY_CORR_URL, sep='|', header=None,
                    names=['STATE', 'ZONE', 'CWA', 'NAME', 'STATE_ZONE',
                           'CZ_NAME', 'FIPS', 'TIME_ZONE', 'FE_AREA', 'LAT', 'LON'])
df_zone_county = df_zone_county[['FIPS', 'LAT', 'LON']]

def clicked(b):
    global df_counties, rec_dict
    year = selYearDrop.value

    output_map.clear_output()
    output_charts.clear_output()
    
    _, df_counties, _ = get_storm_data(year)
    df_counties = df_counties[df_counties['EVENT_TYPE'] == 'Wildfire']
    df_counties.sort_values('TOTAL_DAMAGE', ascending=False, inplace=True)

    lst_options = []
    for i, (_, row) in enumerate(df_counties.iterrows()):
        optionStr = str(row['EVENT_DATE'].month)+'/'+str(row['EVENT_DATE'].day)
        optionStr += '; ' + row['NAME']
        optionStr += '; Damage: ' + f"{int(row['TOTAL_DAMAGE']):,}"
        optionStr += '; fips: ' + row['FIPS']
        lst_options.append(optionStr)

    wildfiresDrop.index = None
    wildfiresDrop.value = None
    wildfiresDrop.options = lst_options

def selEvent(b):
    with output_map:
        year = selYearDrop.value
        
        dirname = WILDFIRE_DATA_URL + str(year) + '/'
        
        output_map.clear_output()
        if b['owner'].description != 'Category':
            output_charts.clear_output()
        
        # Get USDM csv file based on the EVENT_DATE
        # USDM csv files are named with a "date".csv where date is always a Tuesday
        today = df_counties.iloc[wildfiresDrop.index]['EVENT_DATE']
        dirname += today.strftime('%m_%d') + '_' + df_counties.iloc[wildfiresDrop.index]['FIPS']
        
        df_usdm = pd.read_csv(dirname+'/usdm.csv')
       
        df_usdm_merged = df_usdm.merge(df_zone_county, on=['FIPS'])
        df_usdm_merged['NAME'] = df_usdm_merged['County'] + ', ' + df_usdm_merged['State']

        wildfire_county_df = df_usdm_merged[df_usdm_merged['FIPS'] == int(df_counties.iloc[wildfiresDrop.index]['FIPS'])]
        # print(wildfire_county_df.shape, df_counties.iloc[wildfiresDrop.index]['FIPS'])
        df_usdm_merged['FIPS'] = df_usdm_merged['FIPS'].astype(str) # cast it as string in order to zfill
        df_usdm_merged['FIPS'] = df_usdm_merged['FIPS'].apply(lambda x: x.zfill(5)) # no need to zfill

        
        county_idx = wildfire_county_df.iloc[0].name

        #data = df_usdm_merged[['LAT', 'LON']].values.tolist()
        county_lat = df_usdm_merged.iloc[county_idx]['LAT']
        county_lon = df_usdm_merged.iloc[county_idx]['LON']

        counties = get_counties_json()
        
        fig = px.scatter_geo(geojson = counties, 
                             locations=[df_usdm_merged.iloc[county_idx]['FIPS']],
                             hover_name=[df_counties.iloc[wildfiresDrop.index]['NAME']],
                             color_discrete_sequence=['green'],
                             size=[50],
                             basemap_visible = False,
                             scope = 'usa')

        fig_drought = px.choropleth(df_usdm_merged, 
                        geojson = counties,    # use the geo info for counties
                        locations='FIPS',      # location is based on FIPS code
                        hover_name = 'NAME',   # this has both the county and state name
                        hover_data = {'FIPS': False},
                        # center={'lat': county_lat, 'lon': county_lon},
                        color = categoryRadio.value,          # Drought level
                        color_continuous_scale='ylorrd',
                        scope = 'usa') # set the scope to usa to automatically configure the 
                                       # map to display USA-centric data in an appropriate projection.
        fig_drought.add_trace(fig.data[0])
        titleStr = f'Map of {categoryRadio.value} drought conditions<br>'
        titleStr += f"near {df_counties.iloc[wildfiresDrop.index]['NAME']}<br>"
        titleStr += f"on {df_counties.iloc[wildfiresDrop.index]['EVENT_DATE'].strftime('%m-%d')}"
        fig_drought.update_layout(title_text = titleStr)
        fig_drought.show()


    if b['owner'].description == 'Category':
        return # just changed the drought category, no need to update the precipitation data

    with output_charts:
        # Precipitation data
        try:
            df_weather = pd.read_csv(dirname + '/weather.csv')
        except HTTPError as err:
            print(f"No weather data found for {df_counties.iloc[wildfiresDrop.index]['NAME']}")
            return
            
        df_weather.fillna(0, inplace=True)
        df_weather['PRCP'] = df_weather['PRCP'] + df_weather['SNOW']
        df_weather['DATE'] = pd.to_datetime(df_weather['DATE'])
        df_weather_year = df_weather[['DATE','PRCP']].copy()
        df_weather_year['YEAR'] = df_weather['DATE'].apply(lambda r : r.year)
        
        # print(df_weather['TMAX'].max(), df_weather['TMAX'].min())\n",
        q1 = df_weather['TMAX'].quantile(0.25)
        q3 = df_weather['TMAX'].quantile(0.75)
        lower_bound = q1 - 1.5*(q3-q1)
        upper_bound = q3 + 1.5*(q3-q1)
        df_weather['TMAX'].clip(lower_bound, upper_bound, inplace=True)
        
        df_weather_year = df_weather_year.groupby('YEAR', as_index=False).sum()

        line_prcp = alt.Chart(df_weather).mark_line(color='blue', strokeWidth=0.5).transform_window(
            rolling_mean_prcp='mean(PRCP)',
            frame=[-30,0]
        ).encode(
            x = alt.X('DATE:T', axis=alt.Axis(grid=False), title=None),
            y = alt.Y('rolling_mean_prcp:Q', axis=alt.Axis(grid=False), title='PRCP (mm)'),
        ).properties(
            title='30 day rolling average precipitation (mm) for the last 10 years',
            height=150,
            width=800)
        bar_prcp = alt.Chart(df_weather_year).mark_bar(color='blue').encode(
            x = alt.X('YEAR:O', axis=alt.Axis(grid=False), title=None),
            y = alt.Y('PRCP:Q', axis=alt.Axis(grid=False), title='Annual PRCP (mm)'),
        )
        rule = alt.Chart(df_weather_year).mark_rule(color='red').encode(
            y = 'mean(PRCP):Q'
        )
        bar_prcp = (bar_prcp + rule).properties(
            title='Total annuanl PRCP (mm) for the last 10 years with mean',
            height=150,
            width=800
        )
        line_tmin = alt.Chart(df_weather).mark_line(strokeWidth=0.5).transform_window(
            rolling_mean_tmin='mean(TMIN)',
            frame=[-30,0]
        ).encode(
            x = alt.X('DATE:T', axis=alt.Axis(grid=False), title=None),
            y = alt.Y('rolling_mean_tmin:Q', axis=alt.Axis(grid=False), title='TMIN'),
            color=alt.value('blue')
        )
        line_tmax = alt.Chart(df_weather).mark_line(strokeWidth=0.5).transform_window(
            rolling_mean_tmax='mean(TMAX)',
            frame=[-30,0]
        ).encode(
            x = alt.X('DATE:T', axis=alt.Axis(grid=False), title=None),
            y = alt.Y('rolling_mean_tmax:Q', axis=alt.Axis(grid=False), title='TMAX (°C)'),
            color=alt.value('red'), 
        )
        
        #toret = (chart + line).properties(width=800)
        toret = alt.layer(line_tmin, line_tmax).resolve_scale(y='shared').properties(
            title='30 day rolling average min & max temperature (°C) for the last 10 years',
            height=350,
            width=800)
        toret = (line_prcp & bar_prcp & toret).configure_view(strokeOpacity=0)
        #toret = (line_prcp & line_tmin)#.properties(width=800)
        #toret = (line_tmin).properties(width=800)

        toret.display()
        
selYearDrop = widgets.Dropdown(
    description='Select year',
    options=[i for i in get_list_years()],
    value=None
)

categoryRadio = widgets.RadioButtons(description="Category",
                                     options=['D0', 'D1', 'D2', 'D3', 'D4'],
                                     value='D0')

layout_counties = widgets.Layout(width='90%')
wildfiresDrop = widgets.Dropdown(
    value=None, layout=layout_counties
)

wildfireVBox = widgets.VBox([widgets.Label(value='Select county (sorted by damage in $):'), wildfiresDrop])

selYearDrop.observe(clicked, names=['value']) # filter only once, specify value in observe
categoryRadio.observe(selEvent, names=['value'])
wildfiresDrop.observe(selEvent, names=['value'])

list_widgets = [widgets.VBox([widgets.HBox([selYearDrop, categoryRadio]), wildfireVBox])]
accordion = widgets.Accordion(children=list_widgets)
accordion.set_title(0,"Explore WildFire Drought and Weather Data")
display(accordion,res_output)

Accordion(children=(VBox(children=(HBox(children=(Dropdown(description='Select year', options=('2000', '2001',…